# Expert Trajectory Generation

In [1]:
#imports for simulation
from modules.runtime.scenario.scenario_generation.interaction_dataset_scenario_generation import \
    InteractionDatasetScenarioGeneration
from modules.runtime.commons.parameters import ParameterServer
from modules.runtime.viewer.matplotlib_viewer import MPViewer
from modules.runtime.viewer.video_renderer import VideoRenderer
import os
import os.path
import argparse
import matplotlib.pyplot as plt

from IPython.display import clear_output

#imports for observers

#imports for -pkl file


In [2]:
#Set path to interaction dataset
#my current path is /home/marvin/Repositories/interaction_dataset
path_to_interaction_dataset = os.path.join(os.path.expanduser('~'), "Repositories/interaction_dataset")
if not os.path.exists(path_to_interaction_dataset):
    raise ValueError('Interaction dataset not found at location:', path_to_interaction_dataset)

In [3]:
# Lets define the secnario:
param_server = ParameterServer()
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["MapFilename"] = os.path.join(path_to_interaction_dataset, "DR_DEU_Merging_MT/map/DR_DEU_Merging_MT_v01_shifted.xodr")
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["TrackFilename"] = os.path.join(path_to_interaction_dataset, "DR_DEU_Merging_MT/tracks/vehicle_tracks_013.csv")
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["TrackIds"] = [63,64,65,66,67,68]
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["StartTs"] = 232000
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["EndTs"] = 259000
param_server["Scenario"]["Generation"]["InteractionDatasetScenarioGeneration"]["EgoTrackId"] = 66

In [4]:
# Create the scenario
scenario_generation = InteractionDatasetScenarioGeneration(num_scenarios=1, random_seed=0, params=param_server)
scenario = scenario_generation.create_single_scenario()
#following warning will occur:
"""
FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if lane.find("userData"):
"""

/home/marvin/.cache/bazel/_bazel_marvin/d2ffea8a3be3f61c262349ab2b5fa373/execroot/bark_project/bazel-out/k8-fastbuild/bin/docs/tutorials/run.runfiles/bark_project/modules/runtime/commons/xodr_parser.py:105: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if lane.find("userData"):


'\nFutureWarning: The behavior of this method will change in future versions. Use specific \'len(elem)\' or \'elem is not None\' test instead.\n  if lane.find("userData"):\n'

In [5]:
# Initialize
sim_step_time = 0.2
sim_time_steps = 130

In [6]:
# Run the simulation for couple of steps
world_state = scenario.get_world_state()
for _ in range(0, 20):
    world_state.DoPlanning(sim_step_time)
    world_state.DoExecution(sim_step_time)

In [7]:
#gives as all accessible methods for the world_state
print(type(world_state))
dir(world_state)

<class 'bark.world.World'>


['AddAgent',
 'AddEvaluator',
 'AddObject',
 'ClearEvaluators',
 'Copy',
 'DoExecution',
 'DoPlanning',
 'Evaluate',
 'FillWorldFromCarla',
 'GetAgent',
 'GetNearestAgents',
 'GetParams',
 'Observe',
 'PlanAgents',
 'SetMap',
 'Step',
 'UpdateAgentRTree',
 'WorldExecutionAtTime',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'agents',
 'agents_valid',
 'bounding_box',
 'evaluators',
 'map',
 'objects',
 'time']

In [8]:
#get a list of the observed worlds for the agents at the current timestep
list_observed_worlds = world_state.Observe([63,64,65,66,67,68])
#type(list_observed_worlds) : list

In [9]:
#get all methods for an observed world
observed_world=list_observed_worlds[0]
dir(observed_world)

['AddAgent',
 'AddEvaluator',
 'AddObject',
 'ClearEvaluators',
 'Copy',
 'DoExecution',
 'DoPlanning',
 'Evaluate',
 'FillWorldFromCarla',
 'GetAgent',
 'GetAgentBehind',
 'GetAgentInFront',
 'GetNearestAgents',
 'GetParams',
 'Observe',
 'PlanAgents',
 'PredictWithOthersIDM',
 'SetMap',
 'Step',
 'UpdateAgentRTree',
 'WorldExecutionAtTime',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'agents',
 'agents_valid',
 'bounding_box',
 'ego_agent',
 'ego_position',
 'ego_state',
 'evaluators',
 'lane_corridor',
 'map',
 'objects',
 'other_agents',
 'road_corridor',
 'time']

In [13]:
#Unfortunately the method GetEgoAgentID is not accessible for some reasons
#to get the agent id we do a small workaround and go over the ego agent and then take his id
observed_world.ego_agent.id

63

## What does the .pkl file contain?

dict with the keys:
 - "obs" 
 - "next_obs" 
 - "action" 
 - "rew" 
 - "done"
 
every key holds a numpy.ndarray of Shape (rows ,  cols), where the number of
- rows: are the same for all keys
- cols: obs = next_obs = flex , action = flex , rew = done = 1

done is 1 if epoche is finished and 0 otherwise

In [ ]:
"""
for _ in range(0, sim_time_steps):
    world_state.DoPlanning(sim_step_time)
    fig = plt.figure(figsize=[10, 10])
    viewer = MPViewer(params=param_server, use_world_bounds=True, axis=fig.gca())
    viewer.drawWorld(world_state, scenario._eval_agent_ids)
    world_state.DoExecution(sim_step_time)
    clear_output(wait=True)
"""